In [117]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_recommenders as tfrs
import findspark
findspark.init()

In [118]:
os.environ["SPARK_HOME"]=r"D:\spark\spark"
os.environ["HADOOP_HOME"]=r"D:\spark\spark\hadoop\bin"
os.environ["JAVA_HOME"]=r"D:\spark\jdk-21"

In [119]:
jarsql = r"C:\leads\venv\Lib\site-packages\pyspark\mssql-jdbc-9.4.0.jre11.jar"

In [120]:

from pyspark.sql import SparkSession
from os.path import abspath
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, regexp_replace, when, from_unixtime, lit, concat,to_date,unix_timestamp
import pyspark.sql.functions as F

jarsql = r"C:\leads\venv\Lib\site-packages\pyspark\mssql-jdbc-9.4.0.jre11.jar"
warehouse_location = abspath('spark-warehouse')

conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set('spark.driver.extraClassPath', jarsql)
conf.set('spark.executor.extraClassPath', jarsql)
spark = SparkSession.builder\
    .config(conf=conf)\
    .config("spark.sql.warehouse.dir", warehouse_location)\
    .appName("Testing PySpark Example")\
    .getOrCreate()


sc = spark.sparkContext

In [121]:
file_path = r"D:\analise_produtos\excelfiles\super_loja.csv"
df = spark.read.option("sep", ";").option("header", "true").option("inferSchema", "true").csv(file_path)

In [122]:
lista_columns_lower = [i for i in df.columns]
for lista in lista_columns_lower:
    
    df = df.withColumnRenamed(
        lista, lista.replace(" ","_").replace(
            "/","_").replace("�o","ao").replace("a�","ai").lower().strip())
df.show()
    

+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|id_da_fila|  id_do_pedido|data_do_pedido|data_de_envio|modo_de_envio|id_do_cliente|  nome_do_cliente|            segmento|          cidade|          estado|pais_regiao| regiao|  id_do_produto|           categoria|sub-categoria|     nome_do_produto|vendas|quantidade|desconto| lucro|
+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|         1|MX-2021-143658|    02/10/2021|   06/10/2021|Classe padr�o|     SR-20575|      Sofia Rocha|              Varejo|Cidade do M�xico|Distrito

In [123]:
lista_floats = ["vendas","desconto","lucro"]
for col_name in lista_floats:
    df = df.withColumn(col_name, regexp_replace(col(col_name), ",", ".").cast("float"))

In [124]:
date_columns = ["data_do_pedido", "data_de_envio"]
for col_name in date_columns:
    df = df.withColumn(col_name, when(col(col_name) != "NULL", col(col_name)))

In [125]:
date_format = "dd/MM/yyyy"
for col_name in date_columns:
    df = df.withColumn(col_name, to_date(col(col_name), date_format))

In [126]:
tab_vendas = df.select("data_do_pedido","vendas","quantidade","desconto","lucro",
          "data_de_envio","segmento","regiao","categoria","sub-categoria","id_do_produto","nome_do_produto","id_do_pedido","id_do_cliente","estado","cidade")

In [130]:
df = df.withColumn("quantidade", df["quantidade"].cast("int"))
df.show()

dados = df.toPandas()

+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+-------+
|id_da_fila|  id_do_pedido|data_do_pedido|data_de_envio|modo_de_envio|id_do_cliente|  nome_do_cliente|            segmento|          cidade|          estado|pais_regiao| regiao|  id_do_produto|           categoria|sub-categoria|     nome_do_produto|vendas|quantidade|desconto|  lucro|
+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+-------+
|         1|MX-2021-143658|    2021-10-02|   2021-10-06|Classe padr�o|     SR-20575|      Sofia Rocha|              Varejo|Cidade do M�xico|Distr

In [128]:
dados

,id_da_fila,id_do_pedido,data_do_pedido,data_de_envio,modo_de_envio,id_do_cliente,nome_do_cliente,segmento,cidade,estado,pais_regiao,regiao,id_do_produto,categoria,sub-categoria,nome_do_produto,vendas,quantidade,desconto,lucro
0,1,MX-2021-143658,2021-10-02,2021-10-06,Classe padr�o,SR-20575,Sofia Rocha,Varejo,Cidade do M�xico,Distrito Federal,M�xico,Norte,MAT-ET-10001014,Material de escrit�rio,Etiquetas,"HON Etiquetas para organiza��o de pastas, Ajus...",130.800003,3,0.0,45.599998
1,2,MX-2019-155047,2019-10-16,2019-10-21,Classe padr�o,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,Col�mbia,Sul,M�V-M�-10004592,M�veis,M�veis,"Tenex Rel�gio de parede, Dur�vel",2521.600098,8,0.0,907.200012
2,3,MX-2019-155047,2019-10-16,2019-10-21,Classe padr�o,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,Col�mbia,Sul,M�V-ES-10000849,M�veis,Estantes,"Ikea Gaveteiro, Branco",1932.800049,2,0.0,540.799988
3,4,MX-2019-155047,2019-10-16,2019-10-21,Classe padr�o,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,Col�mbia,Sul,MAT-FI-10003284,Material de escrit�rio,Fich�rios,"Cardinal Pasta argolada, Transparente",354.399994,4,0.0,49.599998
4,5,MX-2019-155047,2019-10-16,2019-10-21,Classe padr�o,LC-16570,Larissa Cardoso,Varejo,Dos Quebradas,Risaralda,Col�mbia,Sul,MAT-AR-10002762,Material de escrit�rio,Arte,"Sanford Tela, Tamanhos variados",716.000000,2,0.0,114.400002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,10283,US-2019-145282,2019-04-05,2019-04-08,Primeira classe,LM-17230,Let�cia Martins,Varejo,Ponte Nova,Minas Gerais,Brasil,Sul,MAT-ET-10003909,Material de escrit�rio,Etiquetas,"HON Etiquetas de envio, Ajust�vel",74.160004,3,0.6,-81.839996
10258,10284,US-2019-145282,2019-04-05,2019-04-08,Primeira classe,LM-17230,Let�cia Martins,Varejo,Ponte Nova,Minas Gerais,Brasil,Sul,MAT-EN-10001954,Material de escrit�rio,Envelopes,"Jiffy Fita dupla face, Vermelho",62.799999,1,0.6,-64.400002
10259,10285,US-2019-106523,2019-02-11,2019-02-15,Classe padr�o,RA-19795,Raissa Azevedo,Pequenas e Medias Empresas,S�o Miguel dos Campos,Alagoas,Brasil,Sul,M�V-CA-10004481,M�veis,Cadeiras,"Harbour Creations Cadeira de praia, Vermelho",492.959991,3,0.6,-407.040009
10260,10286,US-2020-168032,2020-11-05,2020-11-09,Classe padr�o,BC-11380,Brenda Cardoso,Grande Empresa,Vassouras,Rio de Janeiro,Brasil,Sul,M�V-ES-10003288,M�veis,Estantes,"Ikea Par, Preto",663.520020,2,0.6,-199.279999


In [134]:
dados = dados.sample(frac=1, random_state=42)
dados["vendas"] = pd.to_numeric(dados["vendas"], errors="coerce").fillna(0)

user_ids = dados["id_do_cliente"].unique()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
dados["id_do_cliente"] = dados["id_do_cliente"].map(user2user_encoded)

produto_ids = dados["id_do_produto"].unique()
produto2produto_encoded = {x: i for i, x in enumerate(produto_ids)}
dados["id_do_produto"] = dados["id_do_produto"].map(produto2produto_encoded)

x = dados[["id_do_cliente", "id_do_produto"]].values
y = dados["vendas"].values
min_sales = dados["vendas"].min()
max_sales = dados["vendas"].max()
y = (y - min_sales) / (max_sales - min_sales)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=42)

In [139]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


def train_model(x_train, y_train, num_users, num_produtos, embedding_size):
    class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_produtos, embedding_size, **kwargs):
            super().__init__(**kwargs)
            self.num_users = num_users
            self.num_produtos = num_produtos
            self.embedding_size = embedding_size
            self.user_embedding = layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.user_bias = layers.Embedding(num_users, 1)
            self.produto_embedding = layers.Embedding(
                num_produtos,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.produto_bias = layers.Embedding(num_produtos, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            produto_vector = self.produto_embedding(inputs[:, 1])
            produto_bias = self.produto_bias(inputs[:, 1])
            dot_user_produto = tf.tensordot(user_vector, produto_vector, 2)
            x = dot_user_produto + user_bias + produto_bias
            return x

    model = RecommenderNet(num_users, num_produtos, embedding_size)
    model.compile(
        loss=keras.losses.MeanSquaredError(),
        optimizer=keras.optimizers.Adam(learning_rate=0.01),
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=64,
        epochs=20,
        verbose=1,
    )

    return model

def get_recommendations_for_all_users(model, num_users, num_recommendations, produto2produto_encoded, user2user_encoded):
    all_recommendations = []

    for user_id in range(num_users):
        user_movie_array = np.array([[user_id] * len(produto2produto_encoded), list(produto2produto_encoded.values())])
        ratings = model.predict(user_movie_array).flatten()
        top_ratings_indices = ratings.argsort()[-num_recommendations:][::-1]
        recommended_products = [{"id_produto": produto_id, "nome_produto": produto2produto_encoded[produto_id], "id_cliente": user_id, "probabilidade": max(0, min(1, ratings[produto_id]))} for produto_id in top_ratings_indices]
        all_recommendations.extend(recommended_products)

    return all_recommendations

num_users = len(user2user_encoded)
num_produtos = len(produto2produto_encoded)
EMBEDDING_SIZE = 64

model = train_model(x_train, y_train, num_users, num_produtos, EMBEDDING_SIZE)
all_recommendations = get_recommendations_for_all_users(model, num_users, 10, produto2produto_encoded, user2user_encoded)

for user_id in range(num_users):
    user_recommendations = [rec for rec in all_recommendations if rec["id_cliente"] == user_id]
    print("Recomendações para o usuário:", user_id)
    print("=" * 40)
    for product in user_recommendations:
        print(f"Código do Produto: {product['id_produto']}, Nome do Produto: {product['nome_produto']}, Probabilidade: {product['probabilidade']:.4f}")
    print()



Epoch 1/20
145/145 [==============================] - 1s 2ms/step - loss: 0.4943
Epoch 2/20
145/145 [==============================] - 0s 2ms/step - loss: 26.4214
Epoch 3/20
145/145 [==============================] - 0s 2ms/step - loss: 185.4966
Epoch 4/20
145/145 [==============================] - 0s 2ms/step - loss: 2626.1641
Epoch 5/20
145/145 [==============================] - 0s 2ms/step - loss: 5344.9336
Epoch 6/20
145/145 [==============================] - 0s 2ms/step - loss: 1203.8914
Epoch 7/20
145/145 [==============================] - 0s 1ms/step - loss: 67.7390
Epoch 8/20
145/145 [==============================] - 0s 2ms/step - loss: 24.4633
Epoch 9/20
145/145 [==============================] - 0s 2ms/step - loss: 25.8048
Epoch 10/20
145/145 [==============================] - 0s 2ms/step - loss: 21.4884
Epoch 11/20
145/145 [==============================] - 0s 3ms/step - loss: 19.7138
Epoch 12/20
145/145 [==============================] - 0s 2ms/step - loss: 17.1273
Epoch 1

In [140]:
def get_recommendations_for_all_users(model, num_users, num_recommendations, produto2produto_encoded, encoded2produto, user2user_encoded):
    all_recommendations = []

    for user_id in range(num_users):
        user_movie_array = np.array([[user_id] * len(produto2produto_encoded), list(produto2produto_encoded.values())])
        ratings = model.predict(user_movie_array).flatten()
        top_ratings_indices = ratings.argsort()[-num_recommendations:][::-1]
        recommended_products = [{"id_do_produto": produto_id, "nome_produto": encoded2produto.get(produto_id, "Nome Desconhecido"),
        "id_do_cliente": user_id, "probabilidade": max(0, min(1, ratings[produto_id]))} for produto_id in top_ratings_indices]
        all_recommendations.extend(recommended_products)

    return all_recommendations


num_users = len(user2user_encoded)
num_produtos = len(produto2produto_encoded)
EMBEDDING_SIZE = 64


""""Obtem Recomendações"""
encoded2produto = {i: x for x, i in produto2produto_encoded.items()}
all_recommendations = get_recommendations_for_all_users(model, num_users, 10, produto2produto_encoded, encoded2produto, user2user_encoded)


"""Exibe Recomendações de produtos"""
for user_id in range(num_users):
    user_recommendations = [rec for rec in all_recommendations if rec["id_do_cliente"] == user_id]
    print("Recomendações para o usuário:", user_id)
    print("=" * 40)
    for product in user_recommendations:
        print(f"Código do Produto: {product['id_do_produto']}, Nome do Produto: {product['nome_produto']}, Probabilidade: {product['probabilidade']:.4f}")
    print()


1/1 [==============================] - 0s 18ms/step
Recomendações para o usuário: 0
Código do Produto: 0, Nome do Produto: 482, Probabilidade: 0.3036
Código do Produto: 1, Nome do Produto: 1804, Probabilidade: 0.2253

Recomendações para o usuário: 1
Código do Produto: 1, Nome do Produto: 1804, Probabilidade: 0.0414
Código do Produto: 0, Nome do Produto: 482, Probabilidade: 0.0000

Recomendações para o usuário: 2
Código do Produto: 1, Nome do Produto: 1804, Probabilidade: 0.7056
Código do Produto: 0, Nome do Produto: 482, Probabilidade: 0.1873

Recomendações para o usuário: 3
Código do Produto: 0, Nome do Produto: 482, Probabilidade: 0.3079
Código do Produto: 1, Nome do Produto: 1804, Probabilidade: 0.1816

Recomendações para o usuário: 4
Código do Produto: 1, Nome do Produto: 1804, Probabilidade: 0.5150
Código do Produto: 0, Nome do Produto: 482, Probabilidade: 0.2175

Recomendações para o usuário: 5
Código do Produto: 0, Nome do Produto: 482, Probabilidade: 0.4823
Código do Produto: 1